In [ ]:
import tarfile
import pyarrow.parquet as pq
##ask about how to use totals.gpus to get gpu_util
metrics = {
    "slurm": ["s21.totals.cpus", "s21.totals.gpus", "s21.totals.memory", "s21.totals.total_nodes"], 
    "ganglia": ["load_five", "Gpu0_gpu_temp", "Gpu1_gpu_temp", "Gpu2_gpu_temp"]  
}

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce


In [ ]:
dfs = []
gpus = []

In [ ]:
with tarfile.open("data/marconi/22-06.tar", "r:*") as tf:
    for metric in metrics["slurm"]:
        alloc = tf.getmember(f"year_month=22-06/plugin=slurm_pub/metric={metric}_alloc/a_0.parquet")
        df_alloc = pq.read_table(tf.extractfile(alloc)).to_pandas()
        df_alloc = df_alloc.groupby("timestamp", as_index=False)["value"].sum()


        config = tf.getmember(f"year_month=22-06/plugin=slurm_pub/metric={metric}_config/a_0.parquet")
        df_config = pq.read_table(tf.extractfile(config)).to_pandas()
        df_config = df_config.groupby("timestamp", as_index=False)["value"].sum()

        util_name = f"{metric.split('.')[-1]} util"
        merged = pd.merge(df_alloc, df_config, on="timestamp", suffixes=("_alloc", "_config"))
        merged[util_name] = (merged["value_alloc"] / merged["value_config"]).clip(0, 1)
        merged["timestamp"] = merged["timestamp"].dt.floor("5min")
        merged = merged.groupby("timestamp", as_index=False)[util_name].mean()
        dfs.append(merged)
        

slurm_final = reduce(lambda l, r: pd.merge(l, r, on="timestamp"), dfs)
slurm_final.head()


In [ ]:
with tarfile.open("data/marconi/22-06.tar", "r:*") as tf:
    ##
    parq = tf.getmember(f"year_month=22-06/plugin=ganglia_pub/metric=load_five/a_0.parquet")
    df = pq.read_table(tf.extractfile(parq)).to_pandas()
    df = df[(df["timestamp"].dt.minute % 5 == 0) & (df["timestamp"].dt.second == 0)]
    df_load_five = df.groupby("timestamp", as_index=False)["value"].sum()
    df_load_five = df_load_five.rename(columns={"value": "Load Average "})

    for metric in metrics["ganglia"][1:]:
        gpux = tf.getmember(f"year_month=22-06/plugin=ganglia_pub/metric={metric}/a_0.parquet")
        gpu_df = pq.read_table(tf.extractfile(gpux)).to_pandas()
        gpu_df["timestamp"] = gpu_df["timestamp"].dt.floor("5min")
        gpu_df = gpu_df.groupby("timestamp", as_index=False)["value"].mean()
        gpu_df = gpu_df.rename(columns={"value": metric})
        gpus.append(gpu_df)

cleaned = reduce(lambda l, r: pd.merge(l, r, on="timestamp"), gpus)        
ganglia_final = pd.merge(cleaned, df_load_five, on="timestamp")
ganglia_final.head()


In [ ]:
Xtrain = pd.merge(ganglia_final, slurm_final, on="timestamp")
Xtrain.head()


In [ ]:
with tarfile.open("data/marconi/22-06.tar", "r:*") as tf:
    m = tf.getmember("year_month=22-06/plugin=logics_pub/metric=Tot_ict/a_0.parquet")
    df_y = pq.read_table(tf.extractfile(m)).to_pandas()
    df_y['timestamp'] = pd.to_datetime(df_y['timestamp'])
    df_y['timestamp'] = df_y['timestamp'].dt.floor('5min')
    df_y = df_y.groupby('timestamp', as_index=False)["value"].mean()

print(df_y.shape)

In [ ]:
df_y['delta'] = df_y['timestamp'].diff().dt.total_seconds().div(60)
gap_threshold = 200   # minutes
gap_idx = df_y.index[df_y['delta'] > gap_threshold][0]
split_time = df_y.loc[gap_idx, 'timestamp']
print(f"Largest gap found: {df_y.loc[gap_idx,'delta']:.1f} minutes at {split_time}")

df_y['post_gap_flag'] = (df_y['timestamp'] >= split_time).astype(int)


In [ ]:
df_y = df_y.drop(columns='delta')
df_y.head()

In [66]:
final_train = pd.merge(df_y, Xtrain, on="timestamp")
df = final_train.copy()

In [72]:
df = df.rename(columns={"value": "P_it"})
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=False)

df['hour'] = df['timestamp'].dt.hour.astype('Int64')
df = df.dropna(subset=['hour'])                        # in case any NaT slipped in
df['hour'] = df['hour'].astype(int)
df['hour_sin'] = np.sin(2*np.pi*df['hour']/24.0)
df['hour_cos'] = np.cos(2*np.pi*df['hour']/24.0)
df = df.drop(columns=["hour"])


X = df.drop(columns=['P_it', 'timestamp']); y = df['P_it']


In [75]:
xtrain, ytrain = X[df['timestamp'] < split_time],  y[df['timestamp'] < split_time]
xtest,  ytest  = X[df['timestamp'] >= split_time], y[df['timestamp'] >= split_time]

n_test = len(xtest)
i_mid  = n_test // 2

xcal, ycal = xtest.iloc[:i_mid], ytest.iloc[:i_mid]

xtest, ytest = xtest.iloc[i_mid:], ytest.iloc[i_mid:]



In [69]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Lasso
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNetCV
import statsmodels.api as sm

In [ ]:
scaler = StandardScaler()

xtrain_scaled = scaler.fit_transform(xtrain)
xcal_scaled   = scaler.transform(xcal)
xtest_scaled  = scaler.transform(xtest)

In [ ]:
##Lasso Model
lasso_cv = LassoCV(alphas=np.logspace(-3,1,100), cv=5)
lasso_cv.fit(xtrain_scaled, ytrain)

y_cal_pred = lasso_cv.predict(xcal_scaled)
bias = ycal.mean() - y_cal_pred.mean()

y_test_lasso = lasso_cv.predict(xtest_scaled) + bias 
mse_lasso = mean_squared_error(ytest, y_test_lasso)
r2_lasso = r2_score(ytest, y_test_lasso)

print(f"Mean Squared Error on test set: {mse_lasso:.2f}")
print(f"  R²   = {r2_lasso:.4f}")
coef_df = pd.Series(lasso_cv.coef_, index=xtrain.columns)
print("\nTop LASSO Coefficients:")
print(coef_df[coef_df != 0].sort_values(ascending=False).head(10))


In [ ]:
##Ridge

ridge_cv = RidgeCV(alphas=np.logspace(-3, 3, 100), cv=5)
ridge_cv.fit(xtrain_scaled, ytrain)


y_cal_ridge = ridge_cv.predict(xcal_scaled)
bias = ycal.mean() - y_cal_ridge.mean()

y_test_ridge = ridge_cv.predict(xtest_scaled) + bias 
mse_ridge = mean_squared_error(ytest, y_test_ridge)
r2_ridge = r2_score(ytest, y_test_ridge)

print(f"Mean Squared Error on test set: {mse_lasso:.2f}")
print(f"  R²   = {r2_lasso:.4f}")
coef_df_ridge = pd.Series(ridge_cv.coef_, index=xtrain.columns)
print("\nTop LASSO Coefficients:")
print(coef_df_ridge[coef_df_ridge != 0].sort_values(ascending=False).head(10))

In [77]:
##Elastic Net
enet_cv = ElasticNetCV(
    alphas=np.logspace(-3, 1, 100), 
    l1_ratio=[.1, .3, .5, .7, .9], 
    cv=5, 
    n_jobs=-1, 
    max_iter=5000
    )
enet_cv.fit(xtrain_scaled, ytrain)

y_cal_enet = enet_cv.predict(xcal_scaled)
bias = ycal.mean() - y_cal_enet.mean()

y_test_enetcv = enet_cv.predict(xtest_scaled) + bias
mse_enet = mean_squared_error(ytest, y_test_enetcv)
r2_enet  = r2_score(ytest, y_test_enetcv)

print(f"Calibration MSE: {mse_enet:.2f}")
print(f"Calibration R²:  {r2_enet:.4f}")



Calibration MSE: 258.42
Calibration R²:  0.5040


In [76]:
model = LinearRegression()

model.fit(xtrain, ytrain)
y_cal_pred  = model.predict(xcal)

bias = ycal.mean() - y_cal_pred.mean()
model.intercept_ += bias
y_test_pred = model.predict(xtest)


rmse_adj = np.sqrt(mean_squared_error(ytest, y_test_pred))
r2_adj   = r2_score(ytest, y_test_pred)

print(f"Adjusted Test RMSE: {rmse_adj:.3f}")
print(f"Adjusted Test R²:   {r2_adj:.3f}")

Adjusted Test RMSE: 16.187
Adjusted Test R²:   0.497


In [ ]:
summary_table = model.summary2().tables[1]
summary_table = summary_table.sort_values("P>|t|")
print("\nTop features by significance (lowest p-values):")
summary_table.head(10)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,5))
plt.plot(ytest.values, label="Actual", alpha=0.8)
plt.plot(y_test_pred, label="Predicted", alpha=0.8)
plt.legend()
plt.title("Test Set: Actual vs Predicted P_IT")
plt.xlabel("Time index")
plt.ylabel("Power (kW)")
plt.show()


In [ ]:
coefs = pd.Series(model.coef_, index=xtest.columns)

beta_sin = coefs.get('hour_sin', 0.0)
beta_cos = coefs.get('hour_cos', 0.0)

hours = np.arange(24)
hour_sin = np.sin(2*np.pi*hours/24.0)
hour_cos = np.cos(2*np.pi*hours/24.0)

# learned diurnal contribution (relative kW)
hour_effect = beta_sin*hour_sin + beta_cos*hour_cos

plt.figure(figsize=(8,3))
plt.plot(hours, hour_effect, marker='o')
plt.xticks(range(0,24,2))
plt.xlabel("Hour of day")
plt.ylabel("Diurnal contribution (kW, relative)")
plt.title("Learned Time-of-Day Effect on P_IT")
plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.show()


In [ ]:
coef_table = pd.DataFrame({
    "Feature": coefs.index,
    "Coefficient (kW per unit)": coefs.values
}).sort_values("Coefficient (kW per unit)", ascending=False)
display(coef_table)
